<a href="https://colab.research.google.com/github/nishJay/taylorGPT/blob/main/taylorGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [48]:
db = pd.read_csv('https://raw.githubusercontent.com/nishJay/taylorGPT/main/taylor_swift_lyrics%20-%20Sheet1%20(1).csv?token=GHSAT0AAAAAAB5FS5SCMVCJ3RGEHSUUR27IY7ONQLA')

In [49]:
db = db.rename(columns={'Unnamed: 2': 'Lyrics'})

In [50]:
db.head()

,Album Title,Song Title,Lyrics
0,Taylor Swift (2006),Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
1,Taylor Swift (2006),Picture to Burn,"State the obvious, I didn't get my perfect fan..."
2,Taylor Swift (2006),Teardrops on My Guitar,"Drew looks at me, I fake a smile so he won't s..."
3,Taylor Swift (2006),A Place in This World,"I don't know what I want, so don't ask me\n'Ca..."
4,Taylor Swift (2006),Cold as You,It's junked out joint off a backroad\nA blinki...


In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt



In [17]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.0 MB/s eta 0:00:00


In [18]:
from transformers import TFGPT2LMHeadModel

In [19]:
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [21]:
from transformers import  GPT2Tokenizer

In [22]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [51]:
db.head()

,Album Title,Song Title,Lyrics
0,Taylor Swift (2006),Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
1,Taylor Swift (2006),Picture to Burn,"State the obvious, I didn't get my perfect fan..."
2,Taylor Swift (2006),Teardrops on My Guitar,"Drew looks at me, I fake a smile so he won't s..."
3,Taylor Swift (2006),A Place in This World,"I don't know what I want, so don't ask me\n'Ca..."
4,Taylor Swift (2006),Cold as You,It's junked out joint off a backroad\nA blinki...


In [52]:
db.describe()

,Album Title,Song Title,Lyrics
count,165,165,165
unique,11,165,165
top,Midnights,Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
freq,23,1,1


In [53]:
train_text = '\n'.join(db['Lyrics'])

In [54]:
train_tokens = tokenizer.encode(train_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (89627 > 1024). Running this sequence through the model will result in indexing errors


In [55]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_tokens)

In [56]:
# Define the sequence length and batch size
sequence_length = 128
batch_size = 16

# Prepare the data for training
def prepare_input_targets(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

train_dataset = train_dataset.batch(sequence_length+1, drop_remainder=True)
train_dataset = train_dataset.map(prepare_input_targets)
train_dataset = train_dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [57]:
# Define the training parameters
epochs = 5
learning_rate = 1e-4

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer])

# Train the model
model.fit(train_dataset, epochs=epochs)

Epoch 1/5
43/43 [==============================] - 1507s 34s/step - loss: 3.2754
Epoch 2/5
43/43 [==============================] - 1464s 34s/step - loss: 2.6177
Epoch 3/5
 8/43 [====>.........................] - ETA: 19:49 - loss: 2.4091

KeyboardInterrupt: ignored